# [Research Request - Explore data for PUC 99314.11 leg report #1613](https://github.com/cal-itp/data-analyses/issues/1613)

Discussed with Peter and Cayman about PUC 99314.11, specifically about sub-section (d)(1)

>(d) (1) On or before November 30, 2025, the department shall submit a report to the Legislature on the revenue vehicle hours, ridership, and passenger mile impacts on the services offered by operators to which Sections 99314.6 and 99314.7 do not apply pursuant to subdivision (a).

1. read in SCO data
2. read in warehouse data for annual ntd reporters

Compare both sources, see which agencies join or not join in either list. 

Compare data in both sources, see if any annual totals match

In [1]:
import pandas as pd


## Read in SCO data

In [2]:
url_sco = "https://bythenumbers.sco.ca.gov/download/bdqr-pszz/application%2Fvnd.openxmlformats-officedocument.spreadsheetml.sheet"

In [3]:
sco_info = pd.ExcelFile(url_sco)
sco_info.sheet_names

['ENTITIES',
 '3 TO_REV_EXP',
 '4 TO_APPROP_LIMIT',
 '5 TO_OPERATING_DATA',
 '6 TO_OPCOST_EXCLUSION_EXEMPT',
 '7 TO_CONTRACT_PRISON',
 '8 TO_LONG_TERM_DEBT',
 '9 TO_STATE_NET_POSIT']

In [4]:
data_sco = pd.read_excel(url_sco, sheet_name="ENTITIES")

In [5]:
display(
    data_sco.info(),
    data_sco.head()
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282 entries, 0 to 281
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Entity Name                 282 non-null    object 
 1   Entity ID                   282 non-null    int64  
 2   Entity Type                 282 non-null    object 
 3   County Name                 281 non-null    object 
 4   Street 1                    265 non-null    object 
 5   Street 2                    85 non-null     object 
 6   P.O. Box                    35 non-null     object 
 7   City                        282 non-null    object 
 8   State                       282 non-null    object 
 9   Zip                         282 non-null    object 
 10  Delete After                3 non-null      float64
 11  Fiscal Year End             282 non-null    int64  
 12  Electronic Report Due Date  282 non-null    int64  
 13  Deleted from System?        282 non

None

,Entity Name,Entity ID,Entity Type,County Name,Street 1,Street 2,P.O. Box,City,State,Zip,Delete After,Fiscal Year End,Electronic Report Due Date,Deleted from System?
0,Access Services for Los Angeles County CTSA - ...,12604,TRS,Los Angeles,3449 Santa Anita Avenue,NaN,NaN,El Monte,CA,91731,NaN,630,131,No
1,Alameda - Specialized Service,443,TRS,Alameda,"2263 Santa Clara, Room 220",NaN,NaN,Alameda,CA,94501,NaN,630,131,No
2,Alameda-Contra Costa Transit District,444,TRS,Alameda,1600 Franklin Street,NaN,NaN,Oakland,CA,94612,NaN,630,131,No
3,Albany - Specialized Service,445,TRS,Alameda,1000 San Pablo Avenue,NaN,NaN,Albany,CA,94706,NaN,630,131,No
4,Alpine County,10443,TRS,Alpine,Alpine County Community Development,50 Diamond Valley Road,NaN,Markleeville,CA,96120,NaN,630,131,No


## read in Warehouse data
